# Multiple Regression(다중회귀)

### Feature engineering(특성공학)
* 기존의 특성을 사용해 새로운 특성을 봅아내는 작업
* 직접만들 수 있지만 sklearn을 사용해 만들어낼 수 있다.

In [11]:
import pandas as pd
import numpy as np

In [12]:
df = pd.read_csv('https://bit.ly/perch_csv_data')
display(df[:5])

perch_full = df.to_numpy()
print(perch_full[:5])

,length,height,width
0,8.4,2.11,1.41
1,13.7,3.53,2.00
2,15.0,3.82,2.43
3,16.2,4.59,2.63
4,17.4,4.59,2.94


[[ 8.4   2.11  1.41]
 [13.7   3.53  2.  ]
 [15.    3.82  2.43]
 [16.2   4.59  2.63]
 [17.4   4.59  2.94]]


In [13]:
perch_weight = np.array(
    [5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0, 110.0,
     115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0, 130.0,
     150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0, 197.0,
     218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0, 514.0,
     556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0, 820.0,
     850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0, 1000.0,
     1000.0]
)

In [14]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(perch_full, perch_weight, random_state=42)

# 사이킷런의 변환기(transformer)

제목 특성공학에서 알 수 있듯, 기존데이터에서 특성을 만들어내야한다.  
이때 하나하나 특성을 만드는 방법도 있겠지만, 사이킷런에서 제공해주는 transformer를 사용해 간단하게 만드는 방법을 배워보자.    

2개의 특성을 가지고있는 [[2, 3]] 데이터를 사용해본다.

In [15]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures()
poly.fit([[2, 3]])
print(poly.transform([[2, 3]]))

[[1. 2. 3. 4. 6. 9.]]


### 결과
[2, 3]을 넣어 6개의 특성을 가진 샘플 [1. 2. 3. 4. 6. 9]을 얻었다.
1. 각 특성
2. 각특성을 제곱한 항
3. 특성끼리 곱한 항
4. 선형 방정식의 절편인 1
    
하지만 사이킷런의 선형 모델은 자동으로 절편을 추가하므로 include_bias=False로 지정하여 1을 없애준다.

In [16]:
poly = PolynomialFeatures(include_bias=False)
poly.fit([[2, 3]])
print(poly.transform([[2, 3]]))

[[2. 3. 4. 6. 9.]]


## 적용

In [33]:
poly = PolynomialFeatures(include_bias=False)
poly.fit(train_input)
train_poly = poly.transform(train_input)
print(train_poly[:2])
print(train_poly.shape)

[[ 19.6      5.14     3.04   384.16   100.744   59.584   26.4196  15.6256
    9.2416]
 [ 22.       5.88     3.52   484.     129.36    77.44    34.5744  20.6976
   12.3904]]
(42, 9)


In [34]:
poly.get_feature_names()

['x0', 'x1', 'x2', 'x0^2', 'x0 x1', 'x0 x2', 'x1^2', 'x1 x2', 'x2^2']

In [35]:
test_poly = poly.transform(test_input)

# 다중 회귀 모델 훈련하기

In [37]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(train_poly, train_target)

LinearRegression()

In [41]:
print(lr.score(train_poly, train_target))

0.9903183436982125


In [40]:
print(lr.score(test_poly, test_target))

0.9714559911594155


# 다음
  
테스트 셋의 점수는 높아지지않았지만 과소적합 문제는 해결되었다.  
여기에서 특성을 조금 더 추가해보자.

In [44]:
poly = PolynomialFeatures(degree=5, include_bias=False)
poly.fit(train_input)
train_poly = poly.transform(train_input)
test_poly = poly.transform(test_input)
print(train_poly.shape)

(42, 55)


In [46]:
lr.fit(train_poly, train_target)
print(lr.score(train_poly, train_target))

0.9999999999938143


In [47]:
print(lr.score(test_poly, test_target))

-144.40744533753661


특성의 개수를 늘려 선형모델이 강력해졌고 훈련세트에 너무 과대적합되어 테스트셋에서는 성능이 나오지않는다.

# 규제

규제(regularization)는 머신러닝 모델이 훈련세트를 너무 과도하게 학습하지 못하도록 훼방하는 것을 말한다. 즉 모델이 훈련 세트에 과대적합되지않게 하기 위함이다.
선형 회귀 모델이 경우 특성에 곱해지는 계수(또는 기울기)이 크기를 작게 만드는 일이다.

---
하지만 이 전에 특성의 스케일을 생각해야된다. -- 2장의 내용  
사이킷런의 StandardScaler 클래스를 사용해 스케일을 맞춘다.

In [49]:
from sklearn.preprocessing import StandardScaler

In [54]:
ss = StandardScaler()
ss.fit(train_poly)
train_scaled = ss.transform(train_poly)
test_scaled = ss.transform(test_poly)

이렇게 선형 회귀 모델에 규제를 추가한 모델을 릿지(ridge)오 라쏘(lasso)라고 부른다.  
두 모델을 규제를 가하는 방법이 다르다.  
* 릿지 : 계수를 제곱한 값을 기준으로 규제를 적용
* 라쏘 : 계수의 절대값을 기준으로 규제를 적용

# 릿지 회귀

In [59]:
from sklearn.linear_model import Ridge

ridge = Ridge()
ridge.fit(train_scaled, train_target)

Ridge()

In [60]:
print(ridge.score(test_scaled, test_target))

0.979069397761539


이렇게 많은 특성을 사용했지만 과대적합되지 않아 테스트 세트에서도 좋은 성능을 내고있다.

In [ ]:
import  matplotlib.pyplot as plt
train_score = []
test_score = []